# Portafolio de Crypto

Acabo de meter mi dinero a Binance, estaba haciendo copy-traiding, pero los que le mueven no saben de esto :p

### Código

In [3]:
import pandas as pd
import requests
import time
import os
from datetime import datetime

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [ ]:
def get_crypto_prices(crypto_id, currency="usd", days=365):
    """
    Descarga precios ajustados históricos de una criptomoneda específica desde CoinGecko.

    Args:
        crypto_id (str): ID de la criptomoneda en CoinGecko (ej. "bitcoin").
        currency (str): Moneda para los precios (por defecto "usd").
        days (int): Número de días de historial (por defecto 30).
    """

    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
    params = {"vs_currency": currency, "days": days, "interval": "daily"}
    response = requests.get(url, params=params)

    # Si la respuesta es exitosa, se procesan los datos
    if response.status_code == 200:

        # Se convierte la respuesta a un DataFrame
        data = response.json()["prices"]
        df = pd.DataFrame(data, columns=["timestamp", "adjusted_price"])
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

        # Se guarda el archivo en la carpeta "crypto_data_YYYY-MM-DD"
        today = datetime.now().strftime("%Y-%m-%d")
        folder = f"crypto_data_{today}_{days}"
        if not os.path.exists(folder):
            os.makedirs(folder)

        filename = f"{folder}/{crypto_id}_prices.csv"
        df.to_csv(filename, index=False)

        print(f"Precios de {crypto_id} guardados en {filename}")        
        return None # No hay errores
    
    
    # Si la respuesta no es exitosa, se imprime un mensaje de error
    else:
        print(f"Error al obtener datos de {crypto_id}: {response.status_code}")
        
        if response.status_code == 429:
            print(f"Demasiadas peticiones. Esperando {response.headers['Retry-After']} segundos.")
            time.sleep(int(response.headers["Retry-After"]))
            get_crypto_prices(crypto_id, currency, days)
            
        return response.status_code

In [ ]:
# Lista de 50 criptomonedas
cryptos = [
    "bitcoin", "ethereum", "tether", "binancecoin", "usd-coin",
    "ripple", "cardano", "solana", "dogecoin", "polkadot",
    "shiba-inu", "dai", "tron", "avalanche-2", "wrapped-bitcoin",
    "uniswap", "litecoin", "chainlink", "cosmos", "stellar",
    "monero", "algorand", "bitcoin-cash", "vechain", "filecoin",
    "internet-computer", "hedera", "decentraland", "the-sandbox", "tezos",
    "elrond-erd-2", "theta-token", "aave", "eos", "pancakeswap-token",
    "flow", "zcash", "maker", "klay-token", "neo",
    "iota", "fantom", "kusama", "waves", "dash",
    "synthetix-network-token", "compound", "chiliz", "enjincoin"
]

In [ ]:
for crypto in cryptos:
    get_crypto_prices(crypto, days=365)

In [1]:
def combine_crypto_csvs(csv_folder):
    """
    Combina múltiples archivos CSV de criptomonedas en un único DataFrame.

    Args:
        csv_folder (str): Carpeta donde se encuentran los archivos CSV.

    Returns:
        pd.DataFrame: DataFrame combinado donde las columnas son los nombres
                      de las criptomonedas y las filas las fechas.
    """
    all_data = {}  # Diccionario para almacenar datos de cada criptomoneda

    for file in os.listdir(csv_folder):
        if file.endswith(".csv"):
            crypto_name = file.replace("_prices.csv", "")  # Obtener nombre de la cripto
            file_path = os.path.join(csv_folder, file)
            df = pd.read_csv(file_path)  # Leer el CSV
            df["timestamp"] = pd.to_datetime(df["timestamp"])  # Asegurar formato datetime
            df = df.set_index("timestamp")  # Usar fecha como índice
            all_data[crypto_name] = df["adjusted_price"]  # Guardar precios ajustados

    # Combinar todos los DataFrames en uno solo
    combined_df = pd.DataFrame(all_data)
    return combined_df

In [4]:
csv_folder = "crypto_data_2024-12-19_365"  # Carpeta donde están los CSV
crypto_prices_df = combine_crypto_csvs(csv_folder)

# Guardar el DataFrame combinado en un archivo CSV
crypto_prices_df.to_csv("combined_crypto_prices.csv")
print("Datos combinados guardados en 'combined_crypto_prices.csv'.")

# Mostrar el DataFrame combinado
print(crypto_prices_df.head())

Datos combinados guardados en 'combined_crypto_prices.csv'.
                  aave  algorand  avalanche-2  binancecoin  bitcoin-cash  \
timestamp                                                                  
2023-12-22  101.455626  0.223064    46.054342   271.073933    233.085976   
2023-12-23  100.128661  0.240550    45.495661   271.431453    238.659824   
2023-12-24   99.410580  0.235473    48.114413   271.254301    233.224955   
2023-12-25   99.848301  0.231077    47.790837   264.652850    229.111577   
2023-12-26  103.189056  0.238222    48.359525   267.178800    234.926906   

                 bitcoin   cardano  chainlink     cosmos       dai  ...  \
timestamp                                                           ...   
2023-12-22  43849.699599  0.636310  15.279277  11.398848  0.999468  ...   
2023-12-23  44003.696022  0.623618  15.497920  11.391916  0.998671  ...   
2023-12-24  43752.030885  0.614803  15.681732  11.314822  0.998909  ...   
2023-12-25  43034.971063  0.5930

In [5]:
def portafolio(data, total_investment: int, obj: str, start_date: str, end_date: str, date: bool, freq: int):

    if date:
        data = data.loc[start_date:end_date]

    mu = expected_returns.mean_historical_return(data, frequency=freq)
    s = risk_models.sample_cov(data, frequency=freq)

    ef = EfficientFrontier(mu, s)
    
    if obj == "max sharpe":
        weights = ef.max_sharpe()

    elif obj == "min volatuility":
        weights = ef.min_volatility()

    elif obj == "max return":
        weights = ef.max_quadratic_utility()

    elif obj == "efficient_risk":
        weights = ef.efficient_risk(0.1)

    elif obj == "efficient_return":
        weights = ef.efficient_return(0.1)

    amounts = ef.clean_weights()

    for crypto, weight in weights.items():
        amounts[crypto] = weight * total_investment

    print(f"Amounts to invest in each crypto to {obj}:")
    for crypto, amount in amounts.items():
        if amount > 0:
            print(f"{crypto}: ${amount:.2f}")

    print("\nExpected return, volatility and Sharpe ratio:")

    print(ef.portfolio_performance(verbose=True))

    return

### Ejemplos

In [6]:
data = crypto_prices_df

In [11]:
portafolio(data, 1000, "max return", "2023-12-14", "2024-12-12", date=False, freq=365)

Amounts to invest in each crypto to max return:
dogecoin: $62.58
ripple: $937.42

Expected return, volatility and Sharpe ratio:
Expected annual return: 224.4%
Annual volatility: 78.1%
Sharpe Ratio: 2.85
(2.244268952512581, 0.7805516651018862, 2.849611437600668)


c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")


In [10]:
portafolio(data, 1000, "max sharpe", "2023-12-14", "2024-11-12", date=False, freq=365)

Amounts to invest in each crypto to max sharpe:
aave: $58.07
binancecoin: $352.15
dogecoin: $68.33
ripple: $445.67
tron: $75.78

Expected return, volatility and Sharpe ratio:
Expected annual return: 178.6%
Annual volatility: 54.7%
Sharpe Ratio: 3.23
(1.785599541169462, 0.5472400993309415, 3.226370917131425)


c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
